In [1]:
from webexteamssdk import WebexTeamsAPI
from webexteamsbot import TeamsBot
import os
import requests
import re, datetime
from datetime import datetime, timedelta 
import pandas as pd
from webexteamssdk import WebexTeamsAPI
import requests
from openpyxl import Workbook
import openpyxl

In [ ]:
#In CMD

:: export TEAMS_BOT_URL=http://77af5e8d.ngrok.io/
:: export TEAMS_BOT_TOKEN=MTg0ZTdhOTMtOTE5NS00NzAxLTgwNWEtZDQwZmM1NmZiODk5ODJjODIxZmEtNmZm_PF84_1eb65fdf-9643-417f-9974-ad72cae0e10f
:: export TEAMS_BOT_EMAIL=timerec@webex.bot
:: export TEAMS_BOT_APP_NAME=Time Recording Bot

In [2]:
bot_email = os.getenv("TEAMS_BOT_EMAIL")
teams_token = os.getenv("TEAMS_BOT_TOKEN")
bot_url = os.getenv("TEAMS_BOT_URL")
bot_app_name = os.getenv("TEAMS_BOT_APP_NAME")

print(bot_app_name)

None


In [ ]:
################################################

In [5]:
bot_email = "timerec@webex.bot"
teams_token = "MTg0ZTdhOTMtOTE5NS00NzAxLTgwNWEtZDQwZmM1NmZiODk5ODJjODIxZmEtNmZm_PF84_1eb65fdf-9643-417f-9974-ad72cae0e10f"
bot_url = "https://c4175134.ngrok.io"
bot_app_name = "Time Recording Bot"
room_ID = "Y2lzY29zcGFyazovL3VzL0FQUExJQ0FUSU9OL2VhMDZiMTMyLWM5YzEtNDY1Zi05OGJmLTVmNzE3Y2RmYTMxYw"

https://teams.webex.com/spaces/Y2lzY29zcGFyazovL3VybjpURUFNOmV1LWNlbnRyYWwtMV9rOmlkZW50aXR5TG9va3VwL1JPT00vNmU3ZjYxMzQtN2VjNy0zMjY1LTg3NTItZmI3YzExMzk4MmQ0

https://teams.webex.com/spaces/Y2lzY29zcGFyazovL3VzL0FQUExJQ0FUSU9OL2VhMDZiMTMyLWM5YzEtNDY1Zi05OGJmLTVmNzE3Y2RmYTMxYw
    
api = WebexTeamsAPI(teams_token)

In [3]:
u = "http://worldclockapi.com/api/json/{timezone}/now".format(timezone="CET")
r = requests.get(u).json()


returned_data = r["currentDateTime"].split("T")
cur_time = returned_data[1]
cur_time = (cur_time.split("+"))[0]
cur_time

'11:39'

In [ ]:
bot = TeamsBot(
    bot_app_name,
    teams_bot_token=teams_token,
    teams_bot_url=bot_url,
    teams_bot_email=bot_email,
)

def greeting(incoming_msg):
    get_infos(incoming_msg)
    return ("Hello {}, I'm your time recording bot. \n\n See what I can do by asking for **/help**.").format(first_name)


def get_infos(incoming_msg):
    '''Function for fetching general Webex Space information'''
    global first_name, room_id, room_name, full_name, filename, filepath
    
    msg_inf = bot.teams.people.get(incoming_msg.personId) #fetching infos
    room_inf = bot.teams.rooms.get(incoming_msg.roomId)
    
    first_name = (str(msg_inf.firstName)) 
    room_id = str(room_inf.id) 
    room_name = str(room_inf.title) 
    full_name = (str(msg_inf.displayName))
    
    if room_name == full_name: #chosing the name for the final file
        filename = first_name
    else:
        filename = room_name    
    filepath = str(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
    
def record_time(incoming_msg):
    return date
    
def date(incoming_msg):
    global cur_date
    
    get_infos(incoming_msg)
    date_test = (incoming_msg.text).replace("/record","")
    date_test = (date_test.replace(" ",""))
    if len(str(date_test)) != 8:
        if len(str(date_test)) > 0:
            reply = "I can: \n\n **/recordtime** for today's date. \n\n**/recordtime DDMMYYYY** for a specific date."
            return reply
            
        else:    
            global cur_date
            u = "http://worldclockapi.com/api/json/{timezone}/now".format(timezone="CET")
            r = requests.get(u).json()

            if r["serviceResponse"]:
                return "Error: " + r["serviceResponse"]

            returned_data = r["currentDateTime"].split("T")
            cur_date = returned_data[0]
            reply = "Recording for {DATE}. \n\n Set start time with **/start HHMM**.".format(DATE=cur_date)
            api.messages.create(roomId=room_id, text=("Records updated with " + str(file_df.shape[0]) + " new entry. " + str(rows) + " total entries."), files=[filepath])
            return
            return reply
    else: 
        cur_date = datetime.strptime(str(date_test), '%d%m%Y').date()
        reply = "Recording for {DATE}. \n\n Set start time with **/start HHMM**.".format(DATE=cur_date)
        return reply
   
    
def start_time(incoming_msg):
    global start_time_str, start_time_final_str, start_time_final
    start_time = (incoming_msg.text).replace("/start ","")
    start_time_str = str(start_time)
    if len(start_time_str) != 3:
        if len(start_time_str) != 4:
            reply = "Please enter a valid time format: \n\n **/start HMM** or **HHMM** f.e.: 1230. \n\n"
        else:
            start_time_final = datetime.strptime(start_time_str, '%H%M')
            start_time_final_str = str(start_time_final.time())[:-3]
            return ("Your start time for " + str(cur_date) + " is: {} \n\n Chose a start time by writing: /end hr.: ").format(start_time_final_str)
    else:
        start_time_final = datetime.strptime(start_time_str, '%H%M')
        start_time_final_str = str(start_time_final.time())[:-3]
        return ("Your start time for " + str(cur_date) + " is: {} \n\n Chose a start time by writing: /end hr.: ").format(start_time_final_str)
                   

def end_time(incoming_msg):
    global end_time_str, end_time_final_str, end_time_final
    end_time = (incoming_msg.text).replace("/end","")
    end_time = (end_time.replace(" ",""))
    end_time_str = str(end_time)
    if len(end_time_str) != 3:
        if len(end_time_str) != 4:
            if len(end_time_str) > 0:
                reply = "Please enter a valid time format: /end HMM or HHMM"
                return reply
            else: 
                u = "http://worldclockapi.com/api/json/{timezone}/now".format(timezone="CET")
                r = requests.get(u).json()

                returned_data = r["currentDateTime"].split("T")
                end_time_final = returned_data[1]
                end_time_final_str = ((end_time_final.split("+"))[0])
                end_time_final = ((end_time_final.split("+"))[0]).replace(":","")
                calc_total(start_time_str, end_time_final)
                reply = "Your end time for " + str(cur_date) + " is: {TIME}.".format(TIME=end_time_final_str) + " You have worked a totel of: " + total_time_str + " hours. \n\n Save with: /save"
                return reply
        else:
            end_time_final = datetime.strptime(end_time_str, '%H%M') #for calc
            end_time_final_str = str(end_time_final.time())[:-3]
            calc_total(start_time_str, end_time_str)
            reply = "Your end time for " + str(cur_date) + " is: {TIME}.".format(TIME=end_time_final_str) + " You have worked a totel of: " + total_time_str + " hours. \n\n Save with: /save"
            return reply
    else:
        end_time_final = datetime.strptime(end_time_str, '%H%M') #for calc
        end_time_final_str = str(end_time_final.time())[:-3]
        calc_total(start_time_str, end_time_str)
        reply = "Your end time for " + str(cur_date) + " is: {TIME}.".format(TIME=end_time_final_str) + " You have worked a totel of: " + total_time_str + " hours. \n\n Save with: /save"
        return reply
    
def calc_total(start, end):
    global total_time, total_time_str
    formatt = '%H%M'
    total_time = datetime.strptime(end, formatt) - datetime.strptime(start, formatt)
    total_time_str = str(total_time)[:-3]   
    
    
def save(incoming_msg):
    get_infos(incoming_msg)
    
    file_df = pd.DataFrame()
    file_df["Date"] = [cur_date]
    file_df["Start time"] = [start_time_final_str]
    file_df["End time"] = [end_time_final_str]
    file_df["Total time"] = [total_time_str]   
    
    test_exist = os.path.exists(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
    if test_exist == True:
        df_orig = pd.read_excel(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
        df_final = df_orig.append(file_df)
        rows = list(df_final.shape)[0]

        timerec_file = openpyxl.Workbook()
        timerec_file.save(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
        writer = pd.ExcelWriter(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx", engine='openpyxl')
        df_final.to_excel(writer, index=False)
        writer.save()  
        filepath = str(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
        api.messages.create(roomId=room_id, text=("Records updated with " + str(file_df.shape[0]) + " new entry. " + str(rows) + " total entries."), files=[filepath])
        return
    
    elif test_exist == False:
        timerec_file = openpyxl.Workbook()
        timerec_file.save(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
        writer = pd.ExcelWriter(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx", engine='openpyxl')
        file_df.to_excel(writer, index=False)
        writer.save()  
        filepath = str(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
        api.messages.create(roomId=room_id, text=("Records updated with " + str(file_df.shape[0]) + " new entry. " + str(rows) + " total entries."), files=[filepath])
        return 
    else:
        return "Error: file could not be saved."
    
def delete(incoming_msg):
    get_infos(incoming_msg)
    delete = (incoming_msg.text).replace("/delete","")
    delete = (delete.replace(" ",""))
    delete_str = str(delete)
    if delete_str == "all":
        get_infos(incoming_msg)
        os.remove(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
        return "Records '" + filename + "_TimeRecording.xlsx' deleted."
    elif delete_str.isdigit() == True:
        df_orig = pd.read_excel(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
        rows = list(df_orig.shape)[0]
        if int(delete_str) < int(rows):       
            df_final = df_orig.drop(df_orig.index[int(delete_str)-1])
            rows = list(df_final.shape)[0]
            timerec_file = openpyxl.Workbook()
            timerec_file.save(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
            writer = pd.ExcelWriter(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx", engine='openpyxl')
            df_final.to_excel(writer, index=False)
            writer.save()  
            filepath = str(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
            api.messages.create(roomId=room_id, text=("Record nr. " + str(delete_str) + " deleted. " + str(rows) + " records left."), files=[filepath])
            return
        else:
            "Error, record nr." + delete_str + " doesn't exist."
    elif delete_str == "last":     
        df_orig = pd.read_excel(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
        df_final = df_orig.head(-1)
        rows = list(df_final.shape)[0]
        timerec_file = openpyxl.Workbook()
        timerec_file.save(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
        writer = pd.ExcelWriter(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx", engine='openpyxl')
        df_final.to_excel(writer, index=False)
        writer.save()  
        filepath = str(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
        api.messages.create(roomId=room_id, text="Welcome to the room!", files=[filepath])
        return "Last record delted. " + rows + " records left."
    else:
        return "I can: \n\n **/delete all** to delete all your records for this space. \n\n **/delete last** to delete the last recorded entry. \n\n **/delete x** to delete a specific record number x."
        
def reset(incoming_msg)           
        
bot.set_help_message("Welcome to the time recording bot! \n\n I understand the following commands:\n")

# Add new commands to the box.
bot.set_greeting(greeting)
bot.add_command("/record", "Initializes Recording Process. \n Leave empty for todays date. Format for specific date: DDMMYYYY.", date)
bot.add_command("/start", "Record start time.", start_time)
bot.add_command("/end", "Record end time.", end_time)
bot.add_command("/save", "Save recorded data to .xlsx file.", save)
bot.add_command("/delete", "Save recorded data to .xlsx file.", delete)
bot.remove_command("/echo")


if __name__ == "__main__":
    # Run Bot
    bot.run(host="127.0.0.1", port="3978")

Teams Bot Email: timerec@webex.bot
Teams Token: REDACTED
Found existing webhook.  Updating it.


 * Serving Flask app "Time Recording Bot" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


Configuring Webhook. 
Webhook ID: Y2lzY29zcGFyazovL3VzL1dFQkhPT0svZTg1NjQ1OWYtOGUyYS00MzA1LTg4MmYtYTM5NmFkNTdhMTQ3
 * Running on http://127.0.0.1:3978/ (Press CTRL+C to quit)
Message from: elandman@cisco.com
Found command: /help
127.0.0.1 - - [22/Feb/2020 17:32:33] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2020 17:32:34] "POST / HTTP/1.1" 200 -


In [160]:
df_orig = pd.read_excel(os.getcwd() + "\\" + filename + "_TimeRecording.xlsx")
rows = list(df_orig.shape)[0]
rows

3

In [ ]:

# Find all rooms that have 'webexteamssdk Demo' in their title
all_rooms = api.rooms.list()
demo_rooms = [room for room in all_rooms if 'webexteamssdk Demo' in room.title]

# Delete all of the demo rooms
for room in demo_rooms:
    api.rooms.delete(room.id)

# Create a new demo room
demo_room = api.rooms.create('webexteamssdk Demo')

# Add people to the new demo room
email_addresses = ["test01@cmlccie.com", "test02@cmlccie.com"]
for email in email_addresses:
    api.memberships.create(demo_room.id, personEmail=email)

# Post a message to the new room, and upload a file
api.messages.create(demo_room.id, text="Welcome to the room!",
                    files=["https://www.webex.com/content/dam/wbx/us/images/dg-integ/teams_icon.png"])